#  Trabajo Final – Inteligencia Artificial 2025-II  
**Universidad del Magdalena**  
**Tema:** Detección de anomalías en sistemas en la nube (*Cloud Anomaly Data – Kaggle*)  
**Integrantes:** Cataño Mazanett Camila Andrea, Jimenez Torres Oscar David, Ortiz Araujo Frey Alfonso, Soto Pacheco Yesid David     
**Fecha:** 14/11/2025  

---

###  Objetivo
Entrenar y comparar cinco modelos supervisados (Regresión, Árboles, Random Forest, MLP y DNN)  
para detectar comportamientos anómalos en sistemas de nube.


In [ ]:
# =======================================
# Importar librerías necesarias
# =======================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from tensorflow.keras import layers

sns.set(style="whitegrid")

print(" Librerías importadas correctamente")


✅ Librerías importadas correctamente


# I. Descripción del problema

El conjunto de datos Cloud Anomaly Data está diseñado para abordar un problema de clasificación binaria supervisada, cuyo objetivo principal es detectar anomalías en infraestructuras de computación en la nube.  
Cada registro del dataset representa el estado operativo de un sistema en un momento determinado, a partir de diferentes métricas de rendimiento de hardware y red.

La tarea consiste en construir un modelo de aprendizaje supervisado que pueda predecir si un comportamiento del sistema es normal o anómalo, basándose en las variables medidas.  
Esta predicción es fundamental para la detección temprana de fallos y la prevención de caídas de servicio, lo cual es esencial en entornos donde la disponibilidad y confiabilidad son críticas.

---

## Descripción del conjunto de datos

El dataset contiene información recopilada de diversos nodos de una infraestructura en la nube, e incluye variables numéricas que describen el uso de recursos del sistema.  
Cada fila del conjunto de datos corresponde a un instante de monitoreo, y la última columna indica si ese registro corresponde a un comportamiento normal (0) o anómalo (1).

Entre las principales variables se encuentran:

| Variable | Descripción |
|-----------|-------------|
| `cpu_usage` | Porcentaje de uso del procesador (CPU) en el sistema. |
| `memory_usage` | Porcentaje de memoria utilizada por el sistema. |
| `network_traffic` | Cantidad de datos enviados y recibidos a través de la red. |
| `power_consumption` | Consumo de energía del sistema en vatios (W). |
| `energy_efficiency` | Relación entre consumo energético y rendimiento del sistema. |
| `execution_time` | Tiempo total de ejecución de las tareas (en segundos). |
| `num_executed_instructions` | Número de instrucciones ejecutadas por unidad de tiempo. |
| `task_type_io` | Indicador binario de tareas de entrada/salida (I/O). |
| `task_type_network` | Indicador binario de tareas relacionadas con la red. |
| `task_status_running` | Indicador de tareas actualmente en ejecución. |
| `task_status_waiting` | Indicador de tareas en espera de ejecución. |
| `task_priority_low` | Indicador de tareas con prioridad baja. |
| `task_priority_medium` | Indicador de tareas con prioridad media. |
| `day` | Día del registro de la observación. |
| `hour` | Hora del día en la que se tomó el registro. |
| `month` | Mes del año correspondiente al registro. |
| `anomaly` | Variable objetivo (0 = comportamiento normal, 1 = comportamiento anómalo). |

---

##  Tamaño y características

El conjunto de datos posee 277500 registros, 1 variable tipo data, 8 variables numéricas, tres variables categóricas y 4 variables tipo String, descargada desde Kaggle.  

---

